In [2]:
# Import required libraries
from bokeh.plotting import figure
from bokeh.io import output_file, show,output_notebook, curdoc
import pandas as pd
# Import column
from bokeh.layouts import column, row, gridplot
#from bokeh.resources import INLINE
# Import ColumnDataSource
from bokeh.models import ColumnDataSource, NumeralTickFormatter, DatetimeTickFormatter, FactorRange, Div
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.palettes import RdBu8
from bokeh.models import ColorBar, BoxAnnotation, PolyAnnotation, Spinner, RangeSlider, DateRangeSlider,Select, CustomJS
from bokeh.palettes import Category10_5
from bokeh.layouts import layout, gridplot

output_notebook()

Loading BokehJS ...

## Adding Layout and Div

In [ ]:
# import data
stocks = pd.read_csv('../data/stocks_cleaned.csv')
stocks['date'] = pd.to_datetime(stocks['date'])
AMZN = stocks[stocks['name'] == 'AMZN']
#display(AMZN)
source = ColumnDataSource(data = AMZN)

# Draw the Fig
fig = figure(x_axis_label="Date", y_axis_label="Stock Price ($)")
fig.line(x="date", y="close",  source=source, color="purple")
box = BoxAnnotation(bottom=AMZN["close"].mean(), fill_color="green", fill_alpha=0.3)
fig.add_layout(box)
fig.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")

#Add a title of "Amazon stock prices vs. average price over the period" to the layout.
title = Div(text="Amazon stock prices vs. average price over the period")

show(layout([title], [fig]))

## Spinner Eidget

In [ ]:
stocks2 = pd.read_csv('../data/stocks_cleaned.csv')
stocks2['date'] = pd.to_datetime(stocks2['date'])
source1 = ColumnDataSource(data = stocks2)

# create an array of five pharmaceutical companies

# Draw the figure
labels = ["ABBR", "JNJ", "LLY", "MRK", "NFX"]
fig = figure(x_axis_label="Volume", y_axis_label="Stock Price ($)")
scatter = fig.scatter(x="volume", y="close", source=source1,  legend_field="name", fill_alpha=0.5, fill_color=factor_cmap("name", palette=Category10_5, factors=labels))
title = Div(text="Pharmaceuticals Stock Performance")
fig.xaxis[0].formatter = NumeralTickFormatter(format="0a")

#Create a spinner titled "Glyph size", with sizes ranging from 1 to 30 pixels, changing by 1 pixel with each click, a default value of 4, and a width of 60.
spinner = Spinner(title="Glyph size", low=1, high=30, step=1, value=4, width=60)

# Complete spinner.js_link(), first passing "value", followed by the scatter's glyph attribute, and lastly specifying "size" as the attribute you wish to modify using the spinner.
spinner.js_link("value", scatter.glyph, "size")

show(layout([title], [spinner, fig]))

## Slider Widget

In [ ]:
# Create two data frame for Ford and General Motors
stocks['market_cap']  = stocks['close'] * stocks['volume']
ford = stocks[stocks['name'] == 'F']
gm = stocks[stocks['name'] == 'GM']


fig = figure(x_axis_label="Stock Price ($)", y_axis_label="Market Cap")
fig.scatter(x=ford["close"], y=ford["market_cap"], legend_label="Ford", fill_color="red", fill_alpha=0.5)
fig.scatter(x=gm["close"], y=gm["market_cap"], legend_label="GM", fill_color="green", fill_alpha=0.5)
fig.yaxis[0].formatter = NumeralTickFormatter(format="$0a")

# Create slider with a title of "Stock Price", the start of 10, end of 47, value matching the start and end values inside parentheses, and a step of 1 for each interaction with the widget.
slider = RangeSlider(title="Stock Price", start=10, end=47, value=(10, 47), step=1)

# Link the slider's "value" to the start of the figure's x_range attribute, assigning zero to the attribute selector
slider.js_link("value", fig.x_range, "start", attr_selector=0)

#Repeat to link the slider's "value" to the end of the figure's x_range attribute, assigning one to the attribute selector.
slider.js_link("value", fig.x_range, "end", attr_selector=1)

show(layout([slider], [fig]))


In [ ]:
from datetime import datetime
stocks1 = pd.read_csv('../data/stocks_cleaned.csv')
stocks1['date'] = pd.to_datetime(stocks1['date'])
#display(stocks['name'].unique())
apple = stocks1[stocks1['name'] == 'AAPL']
netflix = stocks1[stocks1['name'] == 'NFLX']
ibm = stocks1[stocks1['name'] == 'IBM']

earliest_date = stocks1["date"].min()
latest_date = stocks1["date"].max()

fig = figure(x_axis_label="Date", y_axis_label="Price")
fig.line(apple["date"], apple["close"], color="green", legend_label="Apple")
fig.line(netflix["date"], netflix["close"], color="red", legend_label="Netflix")
fig.line(ibm["date"], ibm["close"], color="purple", legend_label="IBM")
fig.legend.location = "top_left"

# Call DateRangeSlider(), assigning "Date" to the title, earliest_date and latest_date to start and end, setting value equal to "2014, 6, 2", "2018, 2, 7", and including an increment of 1 day per interaction with the widget.
slider = DateRangeSlider(title="Date", start=earliest_date, end=latest_date, 
                         value=(datetime(2014,6,2), datetime(2018,6,7)), step=1)

# Call .js_link() twice, linking the widget's "value" to the "Start" and "End" of fig's x_range and setting the attribute selector to 0 and 1 in each call respectively.
slider.js_link("value", fig.x_range, "start", attr_selector=0)
slider.js_link("value", fig.x_range, "end", attr_selector=1)

show(layout([slider], [fig]))

## Select Widget

In [ ]:
from datetime import datetime
stocks4 = pd.read_csv('../data/stocks_cleaned.csv')
stocks4['date'] = pd.to_datetime(stocks4['date'])

# Create three seperate dataframe for Delta Air Lines, Southwest Airlines, and Boeing
boeing = stocks.loc[stocks["name"] == "BA"]
delta = stocks.loc[stocks["name"] == "DAL"]
southwest = stocks.loc[stocks["name"] == "LUV"]
fig = figure(x_axis_label="Date", y_axis_label="Stock Price", 
             x_axis_type="datetime")
boeing_line = fig.line(x=boeing["date"], y=boeing["close"],
                       alpha=0.5)
delta_line = fig.line(x=delta["date"], y=delta["close"], 
                      color="red", alpha=0.5)
sw_line = fig.line(x=southwest["date"], y=southwest["close"], 
                   color="green", alpha=0.5)

# Create the Select widget, assigning options as a list containing "Boeing", "Delta", and "Southwest" (in that order), a value of "Boeing", and a title of "Airline".
# Create Select widget
menu = Select(options=["Boeing", "Delta", "Southwest"], value="Boeing", title="Airline") 

# Set up the widget to implement the callback function on a change in "value"
callback = CustomJS(args=dict(line_1=boeing_line, line_2=delta_line,
                             line_3=sw_line), code="""
line_1.visible = true
line_2.visible = true
line_3.visible = true
if (this.value == "Boeing") {line_2.visible = false
							 line_3.visible = false} 
    else {line_1.visible = false}
if (this.value == "Delta") {line_1.visible = false
							line_3.visible = false} 
    else {line_2.visible = false}
if (this.value == "Southwest") {line_1.visible = false
								line_2.visible = false} 
    else {line_3.visible = false}
""")

menu.js_on_change("value", callback)

show(column(menu, fig))

In [1]:
from datetime import datetime
stocks5 = pd.read_csv('../data/stocks_cleaned.csv')
stocks['market_cap']  = stocks['close'] * stocks['volume']
stocks5['date'] = pd.to_datetime(stocks5['date'])

stocks5 = stocks5[stocks5['name'] == 'EA']
close = ColumnDataSource(data = stocks5)
market_cap = ColumnDataSource(data = stocks5)
volume = ColumnDataSource(data = stocks5)

fig1 = figure(x_axis_label="Date", y_axis_label="Stock Price")
fig2 = figure(x_axis_label="Date", y_axis_label="Market Captial")
fig3 = figure(x_axis_label="Date", y_axis_label="Volume")

close_line = fig1.line(x="date", y="close", color="green", source=close)
market_cap_line = fig2.line(x="date", y="market_cap", color="red", source=market_cap)
volume_line = fig3.line(x="date", y="volume", color="purple", source=volume)
fig1.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")
fig1.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")
fig2.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")
fig3.yaxis[0].formatter = NumeralTickFormatter(format="$0a")
market_cap_line.visible = False
volume_line.visible = False

# Create menu
menu = Select(options=["Close", "Market Cap", "Volume"], value="Close", title="Metric") 
callback = CustomJS(args=dict(plot_one=close, plot_two=market_cap, plot_three=volume, line_1=close_line, line_2=market_cap_line, line_3=volume_line), code="""
plot_one.visible = true
plot_two.visible = true
plot_three.visible = true
line_1.visible = true
line_2.visible = true
line_3.visible = true
if (this.value == "Close") {plot_two.visible = false
                            plot_three.visible = false
                            line_2.visible = false
                            line_3.visible = false}
    else {plot_one.visible = false
          line_1.visible = false} 
if (this.value == "Market Cap") {plot_one.visible = false
                                 plot_three.visible = false
                                 line_1.visible = false
                                 line_3.visible = false} 
    else {plot_two.visible = false
          line_2.visible = false}
if (this.value == "Volume") {plot_one.visible = false
                             plot_two.visible = false
                             line_1.visible = false
                             line_2.visible = false}
    else {plot_three.visible = false
          line_3.visible = false}
""")

# Set up the interaction
menu.js_on_change("value", callback)

# Display the layout
#output_file(filename="stock_metrics.html")
layout = layout([menu],[fig1, fig2, fig3])
show(layout)

NameError: name 'pd' is not defined